# PS4: A Contextual Stochastic Bandit Personal Shopper
Fill me in

## Consumer choice problem
Imagine that a consumer must choose $m$ possible goods (where each good is in a category with $k$ alternatives) $\mathbf{n} = \left\{n_{1},n_{2},\dots,n_{m}\right\}$, where $n_{j}\in\mathbb{R}_{\geq\epsilon}$ is the quantity of good $j$ chosen where $\epsilon>0$, i.e., the consumer must choose at least $\epsilon$ units of any good. Different combinations of goods are _scored_ using a utility function $U:\mathbb{R}^{m}\rightarrow\mathbb{R}$. In this case, let's assume our consumer uses [the Cobb-Douglas](https://en.wikipedia.org/wiki/Cobb%E2%80%93Douglas_production_function) utility:
$$
\begin{align*}
U(\mathbf{n}) = \prod_{i=1}^{m}n_{i}^{\gamma_{i}}
\end{align*}
$$
where $\gamma_{i} = \left\{-1,1\right\}$ denote _user sentiment_ parameters: if $\gamma_{j} = 1$, the good $j$ is preferred, otherwise is $\gamma_{j} = -1$ good $j$ is _not_ preferred. Finally, the choice of goods $n_{1},\dots,n_{m}$ is subject to a budget constraint:
$$
\begin{align*}
\sum_{i=1}^{m}n_{i}C_{i} = B
\end{align*}
$$
where $C_{i}$ is the unit cost of good $i$, and $B$ is the total budget the consumer can spend. The objective of a consumer is to maximize the utility of their choice (the combination of goods) subject to a budget constraint.

## Stochastic Multi-Armed Bandits
In the stochastic multi-armed bandit problem, an agent must choose an action $a$ from the set of all possible actions $\mathcal{A}$, where $\dim\mathcal{A} = K$ during each round $t = 1,2,\dots, T$ of a decision task. The agent chooses action $a\in\mathcal{A}$ and receives a reward $r_{a}$ from the environment, where $r_{a}$ is sampled from some (unknown) distribution $\mathcal{D}_{a}$.

For $t = 1,2,\dots,T$:
1. _Aggregator_: The agent picks an action $a_{t} \in \mathcal{A}$ at time time $t$. How the agent makes this choice is one of the main differences between the different algorithms for solving this problem. 
2. _Adversary_: The agent implements action $a_{t}$ and receives a (random) reward $r_{a}\sim\mathcal{D}_{a}$ where $r_{t}\in\left[0,1\right]$. The distribution $\mathcal{D}_{a}$ is only known to the adversary.
3. The agent updates its _memory_ with the reward and continues to the next decision task. 

The agent is interested in learning the mean of the reward distribution of each arm, $\mu(a) = \mathbb{E}\left[r_{t}\sim\mathcal{D}_{a}\right]$, by experimenting against the world (adversary). 
* __Goal__: The goal of the agent is to maximize the total reward. However, the goal of the algorithm designer is to minimize the _regret_ of the algorithm that the agent uses to choose $a\in\mathcal{A}$.

## Task 1: Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [3]:
include("Include.jl");

First, let's build the `world(...)` function. 
* This function takes the action vector `a::Array{Int64,1}` (the indexes of the goods chosen from each of the $m$ categories), the amount of each good selected from each category from our agent, and returns the reward (utility) associated with selecting this action, i.e., $r\sim\mathcal{D}_{a}$. We'll use [a Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) and the Cobb-Douglas utility to model the rewards.

In [5]:
function world(a::Vector{Int64},n::Vector{Float64}, context::MyBanditConsumerContextModel)::Float64

    # initialize -
    γ = context.γ; # consumer preferences (unknown to bandits)
    σ = context.σ; # noise in utility calculation (unknown to bandits)
    B = context.B; # max budget (unknown to bandits)
    C = context.C; # unit costs of goods (unknown to bandits)
    λ = context.λ; # sensitivity to the budget
    β = context.β; # noise model
    ϵ = 0.001; # min unit required

    # compute the reward for this choice -
    U = 0.0;
    BC = 0.0;
    for i ∈ eachindex(a)
        
        nᵢ = max(ϵ, n[i]); # this is how much of good i to purchase (must be geq ϵ)
        aᵢ = a[i]; # this is which good to purchase in category i -
        Cᵢ = C[i][aᵢ]; # cost of chosen good in category i
        γᵢ = γ[i][aᵢ]; # preference of good in category i
        var = (σ[i][aᵢ])^2; # variance for good i
        βᵢ = β[i][aᵢ]; # uncertainty 

        # update the utility -
        U += (γᵢ*log(nᵢ) + var*rand(βᵢ)); 

        # compute the budget constraint -
        BC += nᵢ*Cᵢ;
    end

    # compute the budget constraint violation -
    Ū = U + λ*max(0.0, (BC-B))^2; # use a penalty method to capture budget constraint

    # return the reward -
    return Ū;
end;

## Task 2: Something will go here.
Fill me in

In [7]:
context = let

    # initialize -

    
end

## Task 2: Evaluation of Algorithms
In this task, we'll implement and run the explore first, $\epsilon$-greedy, and the UCB1 algorithms on our example `K`-arm bandit problem. We'll explain each algorithm and look at its outputs.

Let's start with the explore first algorithm.

### Explore First Algorithm
The agent has $K$ arms, $\mathcal{A} = \left\{1,2,\dots,K\right\}$, and the total number of rounds is $T$. The agent uses the following algorithm to choose which arm to pull during each round:
1. _Initialization_: For each arm $a\in\mathcal{A}$, set $N_{a} = (T/K)^{2/3}\cdot\mathcal{O}\left(\log{T}\right)^{1/3}$ (the number of times we try action $a$).
2. _Exploration_: Play each arm $a\in\mathcal{A}$ for $N_{a}$ rounds and record the rewards. After the exploration phase, select the arm $a^{\star}$ with the highest mean reward (break ties arbitrarily).
3. _Explotation_: Play arm $a^{\star}$ for the remaining rounds.

__Theorem__: The _expected_ regret over $T$ rounds of the _uniform exploration_ algorithm is bounded by $\mathbb{E}\left[R(T)\right]\leq{T}^{2/3}\times\mathcal{O}\left(K\cdot\log{T}\right)^{1/3}$, where $K$ is the number of arms, $T$ is the total number of rounds and $N = (T/K)^{2/3}\cdot\mathcal{O}\left(\log{T}\right)^{1/3}$ is the number of rounds in the exploration phase for each action (choice).

#### Implementation
We've created a custom type for each algorithm with information about the algorithm or problem. In this implementation, we are only storing the number of arms. We then pass that model to a solve method that runs the task.

We create an [instance of the `MyExploreFirstAlgorithmModel` type](src/Types.jl) which holds the number of arms `K` using [a `build(...)` method](src/Factory.jl). To solve the problem, we pass the model [to the `solve(...)` method](src/Bandit.jl), which returns the output from the game.
* __Output__: Each of [the `solve(...)` methods](src/Bandit.jl) returns a results array $\mathbf{U}$ holding the records of the game. The rows of the array are the turns of the decision task, while each column corresponds to the action $a\in\mathcal{A}$. If $u_{t,a}>0$, then action $a$ was chosen in round $t$, and $u_{t,a}$ is the reward received for taking action $a$.

In [10]:
results_explore_first = let

    # initialize -
    K = Dict{Int64,Int64}();
    K[1] = 3; # category 1 has three possible choices
    K[2] = 6; # categorty 2 has six possible choices
    K[3] = 4; # category 4 has four possible choices

    # build model -
    m = build(MyExploreFirstAlgorithmModel, (
        K = K, # number of arms
    ));
    
    # results = solve(m, T = T, world = world, context=context);
    # results;
end

MyExploreFirstAlgorithmModel(Dict(2 => 6, 3 => 4, 1 => 3))